In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import pandas as pd
import numpy as np
import torch

import json
from pathlib import Path

import pyciemss.visuals.vega as vega
from pyciemss.utils import get_tspan


In [22]:
# Load cached data...
tspan = get_tspan(1, 50, 500)

root = Path("..")/"test"/"data/"
prior_samples = vega.tensor_load(root/"prior_samples.json")
intervened_samples = vega.tensor_load(root/"intervened_samples.json")
observed_trajectory = vega.tensor_load(root/"observed_trajectory.json")

data =[(i, dict(Rabbits=observed_trajectory['Rabbits_sol'][0][i*10]))
            for i in range(1,50)]

In [38]:
schema = vega.trajectories(prior_samples, tspan, obs_keys=".*_sol")
vega.ipy_display(schema)